In [1]:
import numpy as np
import pandas as pd

# read return information
df = pd.read_csv('stocks.csv')[['ticker','date','ret']]

# cut the date to year month
df.date = df.date//100

# keep only recent 10 years
df = df[df.date>201000]

# change return to percentage points
df['ret'] = df['ret']*100

# number of stocks in our data
n = 28

In [2]:
# below code to re-construct the data from long form to short form
sample = []
for stock in set(df.ticker):
        x = df[df.ticker==stock].set_index('date').dropna()
        if len(x)>120:
            sample.append(x[['ret']].rename({'ret':stock},axis=1))

df = pd.concat(sample,axis=1)

In [3]:
# initiate equally weighted portfolio
w = [1/n]*n

$ w^t \times V \times w \$

In [4]:
# portfolio variance/std
std = np.sqrt(df.cov().mul(w,axis=0).mul(w,axis=1).sum().sum())
# portfolio return
ret = df.mul(w,axis=1).sum(axis=1).mean()*12

In [5]:
# generate random weights (between 0 and 1) and re-weight them so that total weight=1
w = np.random.rand(n)
w = w / w.sum()

In [6]:
# import math library to solve functions (like solver in Excel)
from scipy.optimize import minimize

In [7]:
def sharpe(w):
    std = np.sqrt(df.cov().mul(w,axis=0).mul(w,axis=1).sum().sum())
    ret = df.mul(w,axis=1).sum(axis=1).mean()*12
    # return negative sharpe ratio, so that we can minimize it (equavalent to maxing sharpe ratio)
    return -(ret-1.6)/std

In [10]:
res = minimize(sharpe, # this is the function to minimise
         w, # initial weights, I set to random weights to begin with
         bounds=[(-1,3)]*n, # bounds of weights, I set between -1 and 3
         constraints=[# a list of constraints to we need to set
             {"type":"eq","fun":lambda w:w.sum()-1}, 
             # each constraint is a dict, the "fun" define a function equal to 0, 
             # the weight sum to 1 in this case
         ]
        )

In [16]:
# the sharpe ratio we obtained
-res.fun

8.474370803614635

In [17]:
# market portfolio weights
res.x

array([ 0.08518165,  0.0568209 ,  0.46086119,  0.06622105,  0.11575887,
       -0.15904063, -0.26234858, -0.3004409 ,  0.20589173, -0.16698075,
       -0.41562716,  0.07982189,  0.18042069,  0.17229424,  0.23616501,
       -0.14872322, -0.03544173,  0.17624663,  0.28634166,  0.14418244,
       -0.06991261, -0.05012159,  0.01175525,  0.22597667,  0.01899566,
       -0.29587965,  0.17056078,  0.21102051])